In [ ]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import os

os.environ["WANDB_DISABLED"] = "true"

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted'),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted'),
    }

dataset = load_dataset("tweets_hate_speech_multilingual")

langs = ["en", "de", "fr"]
dataset = DatasetDict({
    "train": dataset["train"].filter(lambda x: x["lang"] in langs),
    "test": dataset["test"].filter(lambda x: x["lang"] in langs),
})

def map_labels(example):
    label_map = {"negative": 0, "neutral": 1, "positive": 2}
    example["labels"] = label_map[example["sentiment"]]
    return example

dataset = dataset.map(map_labels)

model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

def preprocess_data(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

tokenized_dataset = dataset.map(preprocess_data, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

results = trainer.evaluate()
print("Evaluation Results:", results)

new_texts = [
    "I love this product!",  # English
    "Dieses Produkt ist schrecklich.",  # German
    "C'est un produit moyen.",  # French
]
tokenized_texts = tokenizer(new_texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
predictions = model(**tokenized_texts).logits
predicted_labels = np.argmax(predictions.detach().numpy(), axis=1)

label_map = {0: "negative", 1: "neutral", 2: "positive"}
predicted_sentiments = [label_map[label] for label in predicted_labels]
print("Predicted Sentiments:", predicted_sentiments)


DatasetNotFoundError: Dataset 'tweets_hate_speech_multilingual' doesn't exist on the Hub or cannot be accessed.